In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir", "/user/itv018355/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

## Associating points to the grades in order to calculate Loan Score

In [2]:
spark.conf.set("spark.sql.unacceptable_rated_pts", 0)
spark.conf.set("spark.sql.very_bad_rated_pts", 100)
spark.conf.set("spark.sql.bad_rated_pts", 250)
spark.conf.set("spark.sql.good_rated_pts", 500)
spark.conf.set("spark.sql.very_good_rated_pts", 650)
spark.conf.set("spark.sql.excellent_rated_pts", 800)

In [3]:
spark.conf.set("spark.sql.unacceptable_grade_pts", 750)
spark.conf.set("spark.sql.very_bad_grade pts", 1000)
spark.conf.set("spark.sql.bad_grade_pts", 1500)
spark.conf.set("spark.sql.good_grade_pts", 2000)
spark.conf.set("spark.sql.very_good_grade_pts", 2500)

## Tables required to calculate the loan score
#### customers_new
#### loans
#### loan_repayments
#### loan_def_delinq_new
#### loan_def_detail_rec_enq

In [4]:
bad_customers_data_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema",True) \
.load("/user/itv018355/lendingclubproject/bad/customers_data_csv")

In [5]:
bad_customers_data_df.count()

3157

In [6]:
bad_customers_data_df.createOrReplaceTempView("bad_data_customer")

In [7]:
ph_df = spark.sql("select c.member_id, \
case \
    when p.last_payment_amount < (c.monthly_installment * 0.5) then ${spark.sql.very_bad_rated_pts} \
    when p.last_payment_amount >= (c.monthly_installment * 0.5) and p.last_payment_amount < c.monthly_installment then ${spark.sql.bad_rated_pts} \
    when (p.last_payment_amount = (c.monthly_installment)) then ${spark.sql.good_rated_pts} \
    when p.last_payment_amount > (c.monthly_installment) and p.last_payment_amount <= (c.monthly_installment * 1.50) then ${spark.sql.very_good_rated_pts} \
    when p.last_payment_amount > (c.monthly_installment * 1.50) then ${spark.sql.excellent_rated_pts} \
    else ${spark.sql.unacceptable_rated_pts} \
end as last_payment_pts, \
case \
   when p.total_payment_received >= (c.funded_amount * 0.50) then ${spark.sql.very_good_rated_pts} \
   when p.total_payment_received < (c.funded_amount * 0.50) and p.total_payment_received > 0 then ${spark.sql.good_rated_pts} \
   when p.total_payment_received = 0 or (p.total_payment_received) is null then ${spark.sql.unacceptable_rated_pts} \
   end as total_payment_pts \
from itv018355_lendingclub.loan_repayments p \
inner join itv018355_lendingclub.loans c on c.loan_id = p.loan_id where member_id NOT IN (select member_id from bad_data_customer)")

In [8]:
ph_df.createOrReplaceTempView("ph_pts")

In [9]:
spark.sql("select * from ph_pts")

+--------------------+----------------+-----------------+
|           member_id|last_payment_pts|total_payment_pts|
+--------------------+----------------+-----------------+
|dcec9334e70f1cc95...|             800|              650|
|fc58ca61f51f9dcac...|             500|              650|
|2fb62a6ca51063b11...|             500|              650|
|488268a5531951622...|             800|              650|
|ade6026208e48f5f9...|             500|              650|
|7c8b0ca6acddfaeb1...|             800|              650|
|a707b7fe7c38bad65...|             800|              650|
|1df639cddea30c288...|             800|              650|
|22d67005e12d8726d...|             500|              650|
|009cf312bd46551b4...|             500|              650|
|2d995d383622fa80e...|             500|              650|
|88b97663d2562f239...|             500|              650|
|097d871ba6f21ee37...|             500|              650|
|82e698108e1b8b88a...|             500|              650|
|9e89f004a01479234...|             500|              650|
|28ea534e78570f8d1...|             800|              650|
|27a6888d24e06f6e2...|             500|              650|
|c0d144ae7ab17c793...|             500|              500|
|56d1102515f7d6a32...|             800|              650|
|ba484052baf1e604c...|             800|              650|
+--------------------+----------------+-----------------+
only showing top 20 rows

## Loan score calculation criteria 2: Loan Defaulters History(ldh)

In [10]:
ldh_ph_df = spark.sql(
    "select p.*, \
    CASE \
    WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN d.delinq_2yrs BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN d.delinq_2yrs > 5 OR d.delinq_2yrs IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END AS delinq_pts, \
    CASE \
    WHEN l.pub_rec = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec > 5 OR l.pub_rec IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END AS public_records_pts, \
    CASE \
    WHEN l.pub_rec_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.pub_rec_bankruptcies > 5 OR l.pub_rec_bankruptcies IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END as public_bankruptcies_pts, \
    CASE \
    WHEN l.inq_last_6mths = 0 THEN ${spark.sql.excellent_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts} \
    WHEN l.inq_last_6mths BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts} \
    WHEN l.inq_last_6mths > 5 OR l.inq_last_6mths IS NULL THEN ${spark.sql.unacceptable_rated_pts} \
    END AS enq_pts \
    FROM itv018355_lendingclub.loan_defaulters_detail_rec_enq l \
    INNER JOIN itv018355_lendingclub.loan_defaulters_delinq1 d ON d.member_id = l.member_id  \
    INNER JOIN ph_pts p ON p.member_id = l.member_id where l.member_id NOT IN (select member_id from bad_data_customer)")


In [11]:
ldh_ph_df.createOrReplaceTempView("ldh_ph_pts")

In [12]:
spark.sql("select * from ldh_ph_pts")

+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+
|           member_id|last_payment_pts|total_payment_pts|delinq_pts|public_records_pts|public_bankruptcies_pts|enq_pts|
+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+
|000c8875b71a6b47c...|             800|              650|       250|               800|                    800|    800|
|0012728d9f616bdf2...|             500|              500|       800|               800|                    800|    800|
|00151ece27c7ca280...|             800|              650|       800|               800|                    800|    250|
|003769d7f54c7859e...|             500|              500|       250|               800|                    800|    800|
|0037bb910c0a758f5...|             500|              500|       800|               800|                    800|    800|
|003e1e6cbd2920bbb...|             500|              650|       250|               250|                    250|    800|
|004017b21bd4d6271...|             100|              650|         0|               800|                    800|    800|
|005b4c3db3fce07dc...|             500|              650|       250|               250|                    800|    250|
|00710707c563c2119...|             800|              650|       250|               800|                    800|    800|
|007da79904f69970d...|             800|              650|       250|               800|                    800|    800|
|008515010997a80e1...|             500|              650|       800|               800|                    800|    250|
|009ccbe6fb66324d1...|             500|              650|       800|               800|                    800|    800|
|00d10706191726593...|             800|              650|       800|               800|                    800|    250|
|00da692a960ef39f1...|             800|              650|       800|               800|                    800|    250|
|00eb4bf9db783d8c2...|             250|              650|       800|               800|                    800|    800|
|00ec12e214988d8b3...|             650|              650|       800|               800|                    800|    250|
|00f435a80d0440ece...|             500|              500|       100|               800|                    800|    800|
|00fc2ae3ffb1213e4...|             650|              650|       250|               800|                    800|    800|
|00fc8144cb210ba8c...|             500|              650|       250|               250|                    250|    800|
|0110b7f5f4010cbaf...|             800|              650|       800|               800|                    800|    250|
+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+
only showing top 20 rows

## Criteria 3: financial health(fh)

In [13]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.total_high_credit_limit * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.10) AND l.funded_amount <= (a.total_high_credit_limit * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.20) AND l.funded_amount <= (a.total_high_credit_limit * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.30) AND l.funded_amount <= (a.total_high_credit_limit * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.50) AND l.funded_amount <= (a.total_high_credit_limit * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN itv018355_lendingclub.loans l ON ldef.member_id = l.member_id \
   INNER JOIN itv018355_lendingclub.customers a ON a.member_id = ldef.member_id where ldef.member_id NOT IN (select member_id from bad_data_customer)") 

In [14]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

In [15]:
spark.sql("select * from fh_ldh_ph_pts")

+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+---------------+--------+----------------+---------+
|           member_id|last_payment_pts|total_payment_pts|delinq_pts|public_records_pts|public_bankruptcies_pts|enq_pts|loan_status_pts|home_pts|credit_limit_pts|grade_pts|
+--------------------+----------------+-----------------+----------+------------------+-----------------------+-------+---------------+--------+----------------+---------+
|000c8875b71a6b47c...|             800|              650|       250|               800|                    800|    800|            800|     250|             800|   680.00|
|0012728d9f616bdf2...|             500|              500|       800|               800|                    800|    800|            500|     800|             250|   640.00|
|00151ece27c7ca280...|             800|              650|       800|               800|                    800|    250|            800|     500|             250|   500.00|
|003769d7f54c7859e...|             500|              500|       250|               800|                    800|    800|              0|     250|             800|   100.00|
|0037bb910c0a758f5...|             500|              500|       800|               800|                    800|    800|            500|     500|             800|   450.00|
|003e1e6cbd2920bbb...|             500|              650|       250|               250|                    250|    800|            500|     250|             800|   640.00|
|004017b21bd4d6271...|             100|              650|         0|               800|                    800|    800|            800|     250|             800|   500.00|
|005b4c3db3fce07dc...|             500|              650|       250|               250|                    800|    250|            500|     250|             500|   520.00|
|00710707c563c2119...|             800|              650|       250|               800|                    800|    800|            800|     250|             800|   520.00|
|007da79904f69970d...|             800|              650|       250|               800|                    800|    800|            800|     500|             250|   800.00|
|008515010997a80e1...|             500|              650|       800|               800|                    800|    250|            500|     500|             500|   250.00|
|009ccbe6fb66324d1...|             500|              650|       800|               800|                    800|    800|              0|     250|             800|   250.00|
|00d10706191726593...|             800|              650|       800|               800|                    800|    250|            800|     500|             250|   425.00|
|00da692a960ef39f1...|             800|              650|       800|               800|                    800|    250|            800|     800|             500|     0.00|
|00eb4bf9db783d8c2...|             250|              650|       800|               800|                    800|    800|            800|     250|             800|   552.50|
|00ec12e214988d8b3...|             650|              650|       800|               800|                    800|    250|            800|     500|             650|   680.00|
|00f435a80d0440ece...|             500|              500|       100|               800|                    800|    800|            500|     250|             800|   475.00|
|00fc2ae3ffb1213e4...|             650|              650|       250|               800|                    800|    800|            800|     250|             650|   450.00|
|00fc8144cb210ba8c...|             500|              650|       250|               250|                    250|    800|            500|     250|             800|   500.00|
|0110b7f5f4010cbaf...|             800|              650|       800|               800|                    800|    250|            800|     250|             800|   450.00|
+--------------------+----------------+-----

## Final loan score calculation by considering all 3 criterias with the following %*
#### 1. Payment History = 20%
#### 2. Loan Defaulters = 45%
#### 3. Financial Health = 35%

In [16]:
loan_score = spark.sql("SELECT member_id, \
((last_payment_pts+total_payment_pts)*0.20) as payment_history_pts, \
((delinq_pts + public_records_pts + public_bankruptcies_pts + enq_pts) * 0.45) as defaulters_history_pts, \
((loan_status_pts + home_pts + credit_limit_pts + grade_pts)*0.35) as financial_health_pts \
FROM fh_ldh_ph_pts")

In [17]:
loan_score

+--------------------+-------------------+----------------------+--------------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|
+--------------------+-------------------+----------------------+--------------------+
|000c8875b71a6b47c...|             290.00|               1192.50|            885.5000|
|0012728d9f616bdf2...|             200.00|               1440.00|            766.5000|
|00151ece27c7ca280...|             290.00|               1192.50|            717.5000|
|003769d7f54c7859e...|             200.00|               1192.50|            402.5000|
|0037bb910c0a758f5...|             200.00|               1440.00|            787.5000|
|003e1e6cbd2920bbb...|             230.00|                697.50|            766.5000|
|004017b21bd4d6271...|             150.00|               1080.00|            822.5000|
|005b4c3db3fce07dc...|             230.00|                697.50|            619.5000|
|00710707c563c2119...|             290.00|               1192.50|            829.5000|
|007da79904f69970d...|             290.00|               1192.50|            822.5000|
|008515010997a80e1...|             230.00|               1192.50|            612.5000|
|009ccbe6fb66324d1...|             230.00|               1440.00|            455.0000|
|00d10706191726593...|             290.00|               1192.50|            691.2500|
|00da692a960ef39f1...|             290.00|               1192.50|            735.0000|
|00eb4bf9db783d8c2...|             180.00|               1440.00|            840.8750|
|00ec12e214988d8b3...|             260.00|               1192.50|            920.5000|
|00f435a80d0440ece...|             200.00|               1125.00|            708.7500|
|00fc2ae3ffb1213e4...|             260.00|               1192.50|            752.5000|
|00fc8144cb210ba8c...|             230.00|                697.50|            717.5000|
|0110b7f5f4010cbaf...|             290.00|               1192.50|            805.0000|
+--------------------+-------------------+----------------------+--------------------+
only showing top 20 rows

In [18]:
final_loan_score = loan_score.withColumn('loan_score', loan_score.payment_history_pts + loan_score.defaulters_history_pts + loan_score.financial_health_pts)

In [19]:
final_loan_score.createOrReplaceTempView("loan_score_eval")

In [20]:
spark.sql("select * from loan_score_eval").show(20)

+--------------------+-------------------+----------------------+--------------------+----------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|loan_score|
+--------------------+-------------------+----------------------+--------------------+----------+
|000c8875b71a6b47c...|             290.00|               1192.50|            885.5000| 2368.0000|
|0012728d9f616bdf2...|             200.00|               1440.00|            766.5000| 2406.5000|
|00151ece27c7ca280...|             290.00|               1192.50|            717.5000| 2200.0000|
|003769d7f54c7859e...|             200.00|               1192.50|            402.5000| 1795.0000|
|0037bb910c0a758f5...|             200.00|               1440.00|            787.5000| 2427.5000|
|003e1e6cbd2920bbb...|             230.00|                697.50|            766.5000| 1694.0000|
|004017b21bd4d6271...|             150.00|               1080.00|            822.5000| 2052.5000|
|005b4c3db3fce07dc..

In [21]:

very_good = spark.conf.get("spark.sql.very_good_rated_pts")
good = spark.conf.get("spark.sql.good_rated_pts")
bad = spark.conf.get("spark.sql.bad_rated_pts")
very_bad = spark.conf.get("spark.sql.very_bad_rated_pts")
unacceptable = spark.conf.get("spark.sql.unacceptable_rated_pts")


loan_score_final = spark.sql(f"""
    SELECT ls.*, 
    CASE 
        WHEN loan_score > {very_good} THEN 'A'
        WHEN loan_score <= {very_good} AND loan_score > {good} THEN 'B'
        WHEN loan_score <= {good} AND loan_score > {bad} THEN 'C'
        WHEN loan_score <= {bad} AND loan_score > {very_bad} THEN 'D'
        WHEN loan_score <= {very_bad} AND loan_score > {unacceptable} THEN 'E'
        WHEN loan_score <= {unacceptable} THEN 'F'
    END AS loan_final_grade
    FROM loan_score_eval ls
""")



In [22]:
loan_score_final.createOrReplaceTempView("loan_final_table")

In [23]:
spark.sql("select * from loan_final_table").show(20)

+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|loan_score|loan_final_grade|
+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|000c8875b71a6b47c...|             290.00|               1192.50|            885.5000| 2368.0000|               A|
|0012728d9f616bdf2...|             200.00|               1440.00|            766.5000| 2406.5000|               A|
|00151ece27c7ca280...|             290.00|               1192.50|            717.5000| 2200.0000|               A|
|003769d7f54c7859e...|             200.00|               1192.50|            402.5000| 1795.0000|               A|
|0037bb910c0a758f5...|             200.00|               1440.00|            787.5000| 2427.5000|               A|
|003e1e6cbd2920bbb...|             230.00|                697.50|            766

In [24]:
spark.sql("select * from loan_final_table where loan_final_grade IN ('B')")

+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|           member_id|payment_history_pts|defaulters_history_pts|financial_health_pts|loan_score|loan_final_grade|
+--------------------+-------------------+----------------------+--------------------+----------+----------------+
|68599722e4c505d22...|             200.00|                337.50|             87.5000|  625.0000|               B|
|3a3e7b135cc474146...|             120.00|                247.50|            204.7500|  572.2500|               B|
|21fa4e838e0c53722...|             200.00|                270.00|            154.0000|  624.0000|               B|
|2a1ae0b481e76bf10...|             200.00|                247.50|            175.0000|  622.5000|               B|
|39912961c154e4232...|             200.00|                247.50|            175.0000|  622.5000|               B|
+--------------------+-------------------+----------------------+--------------------+----------+----------------+

In [25]:
spark.sql("select count(*) from loan_final_table")

+--------+
|count(1)|
+--------+
| 1102587|
+--------+

In [ ]:
loan_score_final.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv018355/lendingclubproject/processed/loan_score") \
.save()

In [26]:
loan_score1=spark.read \
.load("/user/itv018355/lendingclubproject/processed/loan_score1")

In [47]:
#spark.stop()

In [27]:
loan_df= spark.sql("select * from loan_score1")

AnalysisException: Table or view not found: loan_score1; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [loan_score1], [], false


In [ ]:
loan_score1.createOrReplaceTempView("loans")

In [ ]:
spark.sql("select * from loans")